In [1]:


import pandas as pd
import itertools

global positions, all_comb

def find_best_teams(df,oprk_cutoff):
    df.sort_values(by='AvgPointsPerGame', ascending=False, inplace=True)
    df.head()

    df = df[df["OPRK"] >= oprk_cutoff]
    max_budget = 500


    df_opt = df.copy()
    df_opt["psratio"] = 0
    df_opt["Flex"] = df['Roster Position'].str.contains('FLEX')
    for i, row in df_opt.iterrows():
        df_opt.loc[(i, "Salary")] /= 100
        df_opt.loc[(i, "psratio")] = df_opt.loc[(i, "AvgPointsPerGame")]/df_opt.loc[(i, "Salary")]
    df_opt = df_opt.drop(['Name + ID', 'Roster Position', 'Game Info', 'TeamAbbrev', 'OPRK', 'Roster Position'], axis=1)
    df_opt = df_opt.set_axis(['pos', 'name', 'id', 'salary', 'point', 'psratio', 'flex'], axis='columns', copy=False)
    df_opt.sort_values(by='psratio', ascending=False, inplace=True)



    #df_opt[df_opt["pos"]=="WR"]



    def total_salary(df):
        return sum(df["salary"])

    def total_points(df):
        return sum(df["point"])



    all_comb = {}
    positions = ['QB', 'RB', 'WR', 'TE', 'DST']
    positions_max = {'QB':1, 'RB':2, 'WR':3, 'TE':1, 'DST':1}

    for pos in positions:
        ids = list(df_opt[df_opt["pos"]==pos]["id"])
        temp_comb = {}
        for comb in itertools.combinations(ids, positions_max[pos]):
            cand = df_opt[df_opt["id"].isin(comb)]
            temp_comb[comb] = ( total_salary(cand), total_points(cand) )
        temp_comb = sorted(temp_comb.items(), key=lambda x:x[1][1], reverse=True)

        all_comb[pos] = []
        min_salary = 99999
        for comb in temp_comb:
            if comb[1][0] < min_salary:
                all_comb[pos].append(comb)
                min_salary = comb[1][0]


    all_teams = {}
    def createTeamCombinationsWithoutFlex(pos, team, total_salary, total_points):
        

        if pos == 5:
            all_teams[tuple(sorted(team))] = (total_salary, total_points)
            return

        for comb in all_comb[positions[pos]]:
            createTeamCombinationsWithoutFlex(pos+1, team+comb[0], total_salary+comb[1][0], total_points+comb[1][1])



    createTeamCombinationsWithoutFlex(0, (), 0, 0)


    df_flex = df_opt[df_opt['flex']==True].copy()
    df_flex.sort_values(by='point', ascending=False, inplace=True)


    best_teams = {}
    for comb in all_teams.keys():
        salary = all_teams[comb][0]
        point = all_teams[comb][1]

        if salary>=max_budget:
            continue

        for i, row in df_flex.iterrows():
            if salary + row['salary'] <= max_budget and not row['id'] in comb:
                best_teams[comb+tuple([row['id']])] = ( salary + row['salary'], point + row['point'])
                break



    best_teams = sorted(best_teams.items(), key=lambda x:x[1][1], reverse=True)

    return best_teams 




In [5]:
import pandas as pd
from itertools import combinations
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript


# Create widgets for file upload and cutoff_oprk input
file_upload = widgets.FileUpload(accept='.xlsx', description='Upload Excel file')
cutoff_oprk = widgets.IntText(value=5, description='OPRK Cutoff')
num_teams = widgets.IntText(value=10, description='Num Teams')

# Define output widgets
output_table = widgets.Output()

def on_button_click(b):

    with output_table:
        clear_output()
        if file_upload.value:
            uploaded_file = list(file_upload.value)[0]  # Access the first uploaded file
            file_content = uploaded_file['content']
            with open(uploaded_file['name'], 'wb') as f:
                f.write(file_content)
            df = pd.read_excel(uploaded_file['name'])
            cutoff = cutoff_oprk.value
            best_teams = find_best_teams(df, cutoff)
            # Display each team dataframe
            for tm in best_teams[:num_teams.value]:
                team_df = df[df["ID"].isin(tm[0])]
                total_points = tm[1][1]
                total_salary = tm[1][0]
                print(f"Total Points = {total_points}, Total Salary = {total_salary*100}")
                display(team_df[["Roster Position","Name","Salary","AvgPointsPerGame","OPRK"]])
        else:
            display(Javascript("alert('Please select an excel filebefore clicking submit.');"))
            return
        
        
# Create a button widget to trigger the team calculation
calculate_button = widgets.Button(description='Calculate Best Teams')
calculate_button.on_click(on_button_click)

# Display widgets
display(file_upload, cutoff_oprk, num_teams,calculate_button)
display(output_table)

FileUpload(value=(), accept='.xlsx', description='Upload Excel file')

IntText(value=5, description='OPRK Cutoff')

IntText(value=10, description='Number Teams')

Button(description='Calculate Best Teams', style=ButtonStyle())

Output()